### 1. Import Libraries

In [1]:
import yfinance as yf
from scipy.optimize import minimize
import numpy as np
import os

### 2. Data Obtention

In [2]:
tickers = yf.Tickers('FEURX FRFZX MEDGX VMIAX')
hist = tickers.history(start='2021-01-01',end='2024-12-31')
print('Data:')
print(hist)

[*********************100%***********************]  4 of 4 completed

Data:
Price      Capital Gains                        Close                       \
Ticker             FEURX FRFZX MEDGX VMIAX      FEURX     FRFZX      MEDGX   
Date                                                                         
2021-01-04           0.0   0.0   0.0   0.0  25.407726  7.126150  12.105045   
2021-01-05           0.0   0.0   0.0   0.0  25.426306  7.133669  12.081510   
2021-01-06           0.0   0.0   0.0   0.0  25.426306  7.148701  12.042281   
2021-01-07           0.0   0.0   0.0   0.0  25.342697  7.163735  12.018748   
2021-01-08           0.0   0.0   0.0   0.0  24.385843  7.178772  12.018748   
...                  ...   ...   ...   ...        ...       ...        ...   
2024-12-23           0.0   0.0   0.0   0.0  26.559999  9.033373  11.844902   
2024-12-24           0.0   0.0   0.0   0.0  26.620001  9.023500  11.844902   
2024-12-26           0.0   0.0   0.0   0.0  26.620001  9.023500  11.844902   
2024-12-27           0.0   0.0   0.0   0.0  26.520000  9.0

In [3]:
adj_close = hist['Close']
print('Adj Close:')
print(adj_close)

Adj Close:
Ticker          FEURX     FRFZX      MEDGX      VMIAX
Date                                                 
2021-01-04  25.407726  7.126150  12.105045  73.909180
2021-01-05  25.426306  7.133669  12.081510  75.671135
2021-01-06  25.426306  7.148701  12.042281  78.934021
2021-01-07  25.342697  7.163735  12.018748  79.474739
2021-01-08  24.385843  7.178772  12.018748  78.952682
...               ...       ...        ...        ...
2024-12-23  26.559999  9.033373  11.844902  96.680000
2024-12-24  26.620001  9.023500  11.844902  97.290001
2024-12-26  26.620001  9.023500  11.844902  97.160004
2024-12-27  26.520000  9.023500  11.835007  96.489998
2024-12-30  26.139999  9.023500  11.844902  95.269997

[1004 rows x 4 columns]


In [4]:
if os.path.exists('data.xlsx'):
    os.remove('data.xlsx')
adj_close.to_excel('data.xlsx')

### 3. Portfolio Analysis

In [5]:
adj_close_values = adj_close.values
print('Adj Close values:')
print(np.round(adj_close_values,4))
print(adj_close_values.shape)

Adj Close values:
[[25.4077  7.1262 12.105  73.9092]
 [25.4263  7.1337 12.0815 75.6711]
 [25.4263  7.1487 12.0423 78.934 ]
 ...
 [26.62    9.0235 11.8449 97.16  ]
 [26.52    9.0235 11.835  96.49  ]
 [26.14    9.0235 11.8449 95.27  ]]
(1004, 4)


In [6]:
R = np.log(adj_close_values[1:] / adj_close_values[:-1])
print('R:')
print(np.round(R*100, 4), '%')

R:
[[ 0.0731  0.1055 -0.1946  2.356 ]
 [ 0.      0.2105 -0.3252  4.2216]
 [-0.3294  0.2101 -0.1956  0.6827]
 ...
 [ 0.      0.      0.     -0.1337]
 [-0.3764  0.     -0.0836 -0.692 ]
 [-1.4432  0.      0.0836 -1.2724]] %


In [7]:
RE = np.mean(R, axis=0)*252
RI = np.std(R, axis=0)*np.sqrt(252)
print('RE:', np.round(RE*100, 4), '%')
print('RI:', np.round(RI*100, 4), '%')

RE: [ 0.7139  5.9309 -0.5458  6.3786] %
RI: [24.7666  3.0984  6.169  19.4995] %


In [8]:
Sharpes = RE / RI
print('Sharpes:', np.round(Sharpes, 4))

Sharpes: [ 0.0288  1.9142 -0.0885  0.3271]


In [9]:
S = np.cov(R, rowvar=False)
print('S:')
print(S*100, '%')

S:
[[0.0243649  0.00017486 0.00170849 0.00917301]
 [0.00017486 0.00038134 0.00035884 0.00062248]
 [0.00170849 0.00035884 0.00151171 0.00162324]
 [0.00917301 0.00062248 0.00162324 0.01510364]] %


In [10]:
corr = np.corrcoef(R, rowvar=False)
print('Correlation:')
print(corr)

Correlation:
[[1.         0.05736469 0.28151191 0.4781772 ]
 [0.05736469 1.         0.47262582 0.25937757]
 [0.28151191 0.47262582 1.         0.33970883]
 [0.4781772  0.25937757 0.33970883 1.        ]]


In [11]:
n_assets = R.shape[1]
weights = np.ones(n_assets) / n_assets
print('Weights:', np.round(weights, 4))
print('Sum of weights:', np.sum(weights))

Weights: [0.25 0.25 0.25 0.25]
Sum of weights: 1.0


In [12]:
def rep(w,r):
    return w @ np.transpose(r)

def varp(w,s):
    return w @ s @ np.transpose(w)

In [13]:
ReP = rep(weights, RE)
varP = varp(weights, S)
RiP = np.sqrt(varP)
SharpeP = ReP / RiP
print('Portafolio pre-optimización:')
print('ReP:', round(ReP*100, 4), '%')
print('varP:', round(varP, 4))
print('RiP:', round(RiP*100, 4), '%')
print('SharpeP:', round(SharpeP, 4))

Portafolio pre-optimización:
ReP: 3.1194 %
varP: 0.0
RiP: 0.6552 %
SharpeP: 4.7611


### 4. Portfolio Optimization

In [14]:
def constr(w):
    return np.sum(w) - 1

num_assets = 4
bounds = [(0, None)] * num_assets 

constraints = [{'type': 'eq', 'fun': constr}]

res = minimize(
    fun=lambda w: varp(w, S),
    x0=weights,
    method='SLSQP',
    bounds=bounds,
    constraints=constraints,
    options={'disp': False, 'maxiter': 1000, 'ftol': 1e-12}
)

print('Pesos optimizados:')
print(np.round(res.x*100, 4), '%')


Pesos optimizados:
[ 0.7906 98.3592  0.8502  0.    ] %


In [15]:
ReP = rep(res.x, RE)
varP = varp(res.x, S)   
RiP = np.sqrt(varP)
SharpeP = ReP / RiP
print('Portafolio post-optimización:')
print('ReP:', round(ReP*100, 4), '%')
print('varP:', round(varP, 4))
print('RiP:', round(RiP*100, 4), '%')
print('SharpeP:', round(SharpeP, 4))

Portafolio post-optimización:
ReP: 5.8346 %
varP: 0.0
RiP: 0.1948 %
SharpeP: 29.9504
